In [1]:
!pip install folium
import folium
import requests
!pip install geopy
import geopy
import numpy as np
import pandas as pd

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


#### Using the read_html function in pandas to create the dataframe

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', flavor='html5lib')

In [4]:
df = df[0]
pc = df[0]
bors = df[1]
neigh = df[2]
pc.drop(0,inplace=True)
bors.drop(0,inplace=True)
neigh.drop(0,inplace=True)

#### Creating a new dataframe to store all columns

In [5]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df_codes = pd.DataFrame(columns=cols)

In [6]:
df_codes['PostalCode'] = pc
df_codes['Borough'] = bors
df_codes['Neighborhood'] = neigh

#### Removing the elements whose boroughs have not been assigned

In [7]:
df_codes.drop(df_codes[df_codes.Borough == "Not assigned"].index, inplace=True)

In [8]:
df_codes = df_codes.reset_index()

In [9]:
df_codes.drop(['index'], axis=1, inplace=True)

In [10]:
df_codes.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Copying the borough name to the neighborhood when it is not assigned

In [11]:
for i in range(len(df_codes)):
    if (df_codes['Neighborhood'][i] == "Not assigned"):
        df_codes['Neighborhood'][i] = df_codes['Borough'][i]
df_codes.head(10)        

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Loop for grouping by postalcodes and merging same postalcode cells
##### Inds stores the indexes of the rows merged(which will be dropped later)

In [12]:
inds = []
for i in range(0,(len(df_codes)-1)):
    if( str(df_codes['PostalCode'].iloc[i]) == str(df_codes['PostalCode'].iloc[i+1])):
        df_codes['Neighborhood'].iloc[i] += ', '
        df_codes['Neighborhood'].iloc[i] += df_codes['Neighborhood'].iloc[i+1]
        inds.append(int(i+1))
df_codes.head(10)        

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
9,M1B,Scarborough,Malvern


In [13]:
df_codes.drop(inds, inplace=True)

In [14]:
df_codes = df_codes.reset_index()
df_codes.drop(['index'], axis=1, inplace=True)
df_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [15]:
df_codes.shape

(103, 3)

### Using the csv file (for latitudes and longitudes) because geocoder could not get all values 

In [37]:
df2 = pd.read_csv('D:\DataSets\Geospatial_Coordinates.csv')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


The following cell gives the index of the row satisfying a given condition

In [58]:
df2[df2['Postal Code'] == 'M4A'].index.values.astype(int)[0]

34

As df2 and df_codes are not arranged in the same order, therefore we have to find the index of the row in df2 having the same PostalCode as that in df_codes

In [52]:
for i in range(len(df_codes)):
    ind = df2[df2['Postal Code'] == str(df_codes['PostalCode'].iloc[i])].index.values.astype(int)[0]
    df_codes['Latitude'].iloc[i] = float(df2['Latitude'].iloc[ind])
    df_codes['Longitude'].iloc[i] = float(df2['Longitude'].iloc[ind])

C:\Python27\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [53]:
df_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
